# Sample load creds from env
ala https://12factor.net

In [1]:
import os
from envbash import load_envbash
import psycopg2 as pg
from psycopg2.extras import RealDictCursor
import pandas as pd

In [13]:
%matplotlib inline
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

In [ ]:
init_notebook(connected=True)
# update to use cufflinks offline
cf.go_offline()

In [2]:
# OS agnostic ref to personal cred file if stored in ~/Dropbox/creds/app-env
os.chdir(os.path.join(os.environ["HOME"],"Dropbox","creds"))

In [3]:
load_envbash("app-env")

In [4]:
conn_dict = {"dbname":os.environ["RH_DB"],
             "user":os.environ["RH_USER"],
             "password":os.environ["RH_PWD"],
             "host":os.environ["RH_HOST"]}

In [5]:
conn = pg.connect(**conn_dict)

In [6]:
cur = conn.cursor(cursor_factory=RealDictCursor)
cs_types = "(1)"
query = "select collection_set_id,collection_set,collection_set_status_id,collection_type_id from md2.vi_collection_sets where product_period_id = 25 and collection_set not ilike '%-OPS_%' and collection_set_status_id in (11,20) and collection_type_id in " + cs_types;
cur.execute(query)
markets = cur.fetchall()
cur.close()
conn.close()

In [7]:
df = pd.DataFrame(markets)

In [8]:
df.head()

,collection_set_id,collection_set,collection_set_status_id,collection_type_id
0,8683,NewYork-NY-Leeloo-Drive_2019-1H,11,1
1,8676,Chicago-IL-Leeloo-Drive_2019-1H,11,1
2,8663,Seattle-WA-Leeloo-Drive_2019-1H,11,1
3,8650,SanAntonio-TX_2019-1H,20,1
4,8642,Greenville-SC_2019-1H,20,1


In [9]:
conn = pg.connect(**conn_dict)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = "select * from reporting.rs_core_test_data_1h19 where collection_set_id = 8650";
cur.execute(query)
data = cur.fetchall()
cur.close()
conn.close()

In [10]:
mdf = pd.DataFrame(data)

In [11]:
mdf[0:2].T

,0,1
collection_set_id,8650,8650
collection_area_id,36,36
collection_area,"San Antonio, TX","San Antonio, TX"
collection_set,SanAntonio-TX,SanAntonio-TX
period,2019-1H,2019-1H
carrier,AT&T,AT&T
to_carrier,Sprint,None
test,sms,download
test_event_id,123115562,123180586
device_imei,None,None


In [20]:
ssdls = mdf[(mdf['carrier']=='Verizon') & (mdf['test']=='download')]

In [22]:
ssdls.iplot(kind='scatter', x='lte_rsrp', y='tv_50p', mode='markers')

/Users/edluschei/GitHub/100daysofcode-with-python-course/venv/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

